# librerias y funciones adicionales

In [1]:
from sklearn.metrics import confusion_matrix
import itertools
import pickle

In [2]:
!pip install kora -q
from kora import drive
drive.link_nbs()

In [13]:
from PSOPR import *

importing Jupyter notebook from /nbs/PSOPR.ipynb


In [4]:
def obtener_centroides(datos,etiquetas):
  n = len(np.unique(etiquetas))
  centroides = np.array(
      [np.mean(datos[etiquetas == i],axis=0) for i in range(n)] )
  return(centroides)

In [5]:
def error_rate(y_true,y_pred):
  '''devuelve el porcentaje de datos mal etiquetados, como lo
  calculamos basandonos en la matriz de confusion, debemos considerar
  todas las formas en que pudieron haber sido colocadas las etiqueas
  (que cada grupo tuviera la clase 0,1,2 etc) y se devuelve el error
  mas pequeño, pues se considera esa puntuacion corresponde
  al etiquetado correcto'''
  n = len(y_true)
  indices_grupos = []
  grupos = np.unique(y_pred)
  #generamos las posibles etiquetas de cada grupo
  etiquetados = list(itertools.permutations(grupos))
  errores = []
  #sacamos los indices de los elementos de cada grupo
  for i in range(len(grupos)):
    indices_grupos.append(np.where(y_pred == i))
  #obtenemos el error de cada etiquetado posible
  for etiquetado in etiquetados:
    for clase,etiqueta in enumerate(etiquetado):
      y_pred[indices_grupos[clase] ] = etiqueta
    matriz = confusion_matrix(y_true, y_pred)
    aciertos = np.sum([matriz[i][i] for i in range(len(matriz))])
    errores.append((n - aciertos)/n)
  return(min(errores))

In [6]:
def min_mse(y_true,y_pred):
  n = len(y_true) #numero de datos a predecir
  indices_grupos=[]#guarda la posicion de todos los elemntos de cada grupo
  grupos = np.unique(y_pred) #sacamos las clases encontradas
  errores = []
  #obtenemos todas las formas de etiquetar los grupos en base a las clases
  etiquetados = list(itertools.permutations(grupos))
  #sacamos los indices de los elementos de cada grupo
  for i in range(len(grupos)):
    indices_grupos.append(np.where(y_pred == i))
  #sacamos el mse de cada etiquetado
  for etiquetado in etiquetados:
    for clase,etiqueta in enumerate(etiquetado):
      y_pred[indices_grupos[clase] ] = etiqueta
    errores.append(mse(y_true,y_pred))
  return(min(errores))

In [7]:
def mse_centros(reales,predichos):
  dim = len(reales) #numero de datos a predecir
  grupos = list(range(len(reales))) #guarda la posicion de todos los elemntos de cada grupo
  errores = []
  #obtenemos todas las formas de etiquetar los grupos en base a las clases
  etiquetados = list(itertools.permutations(grupos))
  for etiquetado in etiquetados:
    error = []
    for clase,etiqueta in enumerate(etiquetado):
      error.append(mse(reales[clase],predichos[etiqueta])*dim)
    errores.append(np.mean(error))
  #print(f'errores {errores}')
  return(min(errores))

In [8]:
def analisis_estadistico(datos,x_min,x_max,labels_reales,f_apt,iteraciones):
  errores = []
  lista_mse = []
  aptitudes = []
  mse_centroides = []
  aris = []
  completitudes = []
  k = len(np.unique(labels_reales))
  centros_reales = obtener_centroides(datos,labels_reales)
  metricas = ["error rate","mse","aptitud","mse centros"]
  for i in range(iteraciones):
    res = pso(x_min = x_min, x_max = x_max,v_min = 0,
    v_max=1, n_part= 10,iter_pso=100,iter_kmean=50,c1=2,
    c2=2,datos= datos, k = k ,f_apt = f_apt)
    errores.append(error_rate(labels_reales,res.labels_))
    lista_mse.append(min_mse(labels_reales,res.labels_) )
    aptitudes.append(res.aptitud)
    mse_centroides.append(mse_centros(centros_reales,res.cluster_centers_))
    aris.append(ari(labels_reales,res.labels_))
    completitudes.append(completitud(labels_reales,res.labels_))
  print(f'tras {iteraciones} iteraciones obetenemos lo siguiente')
  print(f'metrica mejor peor media mediana desviacion ')
  for nombre,metrica in zip(metricas,(errores,lista_mse,aptitudes,mse_centroides)):
    print(f'{nombre} {min(metrica):.3f} {np.max(metrica):.3f} {np.mean(metrica):.3f} {np.median(metrica):.4f} {np.std(metrica):.3f}')
  for nombre,metrica in zip(["ARI","completitud"],(aris,completitudes)):
    print(f'{nombre} {max(metrica):.3f} {np.min(metrica):.3f} {np.mean(metrica):.3f} {np.median(metrica):.4f} {np.std(metrica):.3f}')

In [9]:
def recuperar(archivo):
  with open(archivo, "rb") as f:
      recuperado = pickle.load(f)
  return(recuperado)

# Replicar resultados

## Agrega la ruta a los archivos "replica"

In [10]:
path = '/content' #aqui pon la ruta a la carpeta con los archivos replica

## En el conjunto iris

In [11]:
iris_datos,iris_min,iris_max,iris_labels = recuperar(path+'/replicaIris.pickle')

In [ ]:
#replica los resultados usando la función aptitud original
analisis_estadistico(datos=iris_datos,
                     x_min =iris_min,x_max=iris_max,
                     labels_reales=iris_labels,
                     f_apt=aptitud,iteraciones=31)

In [ ]:
#replica los resultados usando la función aptitud DB
analisis_estadistico(datos=iris_datos,
                     x_min =iris_min,x_max=iris_max,
                     labels_reales=iris_labels,
                     f_apt=aptitud2,iteraciones=31)

## En el conjunto vino

In [ ]:
vino_datos,vino_min,vino_max,vino_labels = recuperar(path+'/replicaVino.pickle')

In [ ]:
#replica los resultados usando la función aptitud original
analisis_estadistico(datos=vino_datos,
                     x_min =vino_min,x_max=vino_max,
                     labels_reales=vino_labels,
                     f_apt=aptitud,iteraciones=31)

In [ ]:
#replica los resultados usando la función aptitud DB
analisis_estadistico(datos=vino_datos,
                     x_min =vino_min,x_max=vino_max,
                     labels_reales=vino_labels,
                     f_apt=aptitud2,iteraciones=31)

## En el conjunto Vidrio

In [ ]:
vidrio_datos,vidrio_min,vidrio_max,vidrio_labels = recuperar(path+'/replicaVidrio.pickle')

In [ ]:
#replica los resultados usando la función aptitud original
analisis_estadistico(datos=vidrio_datos,
                     x_min =vidrio_min,x_max=vidrio_max,
                     labels_reales=vidrio_labels,
                     f_apt=aptitud,iteraciones=31)

In [ ]:
#replica los resultados usando la función aptitud DB
analisis_estadistico(datos=vidrio_datos,
                     x_min =vidrio_min,x_max=vidrio_max,
                     labels_reales=vidrio_labels,
                     f_apt=aptitud2,iteraciones=31)

## En el conjunto cancer

In [ ]:
cancer_datos,cancer_min,cancer_max,cancer_labels = recuperar(path+'/replicaCancer.pickle')

In [ ]:
#replica los resultados usando la función aptitud original
analisis_estadistico(datos=cancer_datos,
                     x_min =cancer_min,x_max=cancer_max,
                     labels_reales=cancer_labels,
                     f_apt=aptitud,iteraciones=31)

In [ ]:
#replica los resultados usando la función aptitud DB
analisis_estadistico(datos=cancer_datos,
                     x_min =cancer_min,x_max=cancer_max,
                     labels_reales=cancer_labels,
                     f_apt=aptitud2,iteraciones=31)